In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using =>", device)

In [ ]:
root_dir = "/content/gdrive/My Drive/Colab Notebooks/melanoma_classification/siim-isic-melanoma-classification/torch_jpeg/"
print("The data lies here =>", root_dir)

In [ ]:
image_transforms = {
    "train": transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor()
    ]),
    "test": transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor()
    ])
        }

In [ ]:
melanoma_dataset = datasets.ImageFolder(root = root_dir + "train",
                                      transform = image_transforms["train"]
                                     )
melanoma_dataset

In [ ]:
melanoma_dataset.class_to_idx

In [ ]:
idx2class = {v: k for k, v in melanoma_dataset.class_to_idx.items()}

In [ ]:
# def get_class_distribution(dataset_obj):
#     count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
#     for _, label_id in dataset_obj:
#       label = idx2class[label_id]
#       count_dict[label] += 1
#     return count_dict
# def plot_from_dict(dict_obj, plot_title, **kwargs):
#     return sns.barplot(data = pd.DataFrame.from_dict([dict_obj]).melt(), x = "variable", y="value", hue="variable", **kwargs).set_title(plot_title)
# plt.figure(figsize=(15,8))
# plot_from_dict(get_class_distribution(melanoma_dataset), plot_title="Entire Dataset (before train/val/test split)")

In [ ]:
melanoma_dataset_size = len(melanoma_dataset)
melanoma_dataset_indices = list(range(melanoma_dataset_size))

In [ ]:
np.random.shuffle(melanoma_dataset_indices)

In [ ]:
val_split_index = int(np.floor(0.20 * melanoma_dataset_size))

In [ ]:
train_idx, val_idx = melanoma_dataset_indices[val_split_index:], melanoma_dataset_indices[:val_split_index]

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [ ]:
melanoma_dataset_test = datasets.ImageFolder(root = root_dir + "test",
                                            transform = image_transforms["test"]
                                           )
melanoma_dataset_test

In [ ]:
train_loader = DataLoader(dataset=melanoma_dataset, shuffle=False, batch_size=500, sampler=train_sampler)
val_loader = DataLoader(dataset=melanoma_dataset, shuffle=False, batch_size=100, sampler=val_sampler)
test_loader = DataLoader(dataset=melanoma_dataset_test, shuffle=False, batch_size=80)


In [ ]:
# def get_class_distribution_loaders(dataloader_obj, dataset_obj):
#     count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
#     if dataloader_obj.batch_size == 1: 
#    # if dataloader_obj.batch_size == 32: 
#         for _,label_id in dataloader_obj:
#             y_idx = label_id.item()
#             y_lbl = idx2class[y_idx]
#             count_dict[str(y_lbl)] += 1
#     else: 
#         for _,label_id in dataloader_obj:
#             for idx in label_id:
#                 y_idx = idx.item()
#                 y_lbl = idx2class[y_idx]
#                 count_dict[str(y_lbl)] += 1
#     return count_dict

In [ ]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,7))
# plot_from_dict(get_class_distribution_loaders(train_loader, melanoma_dataset), plot_title="Train Set", ax=axes[0])
# plot_from_dict(get_class_distribution_loaders(val_loader, melanoma_dataset), plot_title="Val Set", ax=axes[1])

In [ ]:
single_batch = next(iter(train_loader))

In [ ]:
single_batch[0].shape

In [ ]:
print("Output label tensors: ", single_batch[1])
print("\nOutput label tensor shape: ", single_batch[1].shape)

In [ ]:
# Selecting the first image tensor from the batch. 
single_image = single_batch[0][0]
single_image.shape

In [ ]:
#plt.imshow(single_image.(1, 2, 0))

plt.imshow(single_image.permute(1, 2, 0))

In [ ]:
# We do single_batch[0] because each batch is a list 
# where the 0th index is the image tensor and 1st index is the output label.
single_batch_grid = utils.make_grid(single_batch[0], nrow=4)
plt.figure(figsize = (10,10))
plt.imshow(single_batch_grid.permute(1, 2, 0))

In [ ]:
class melanomaClassifier(nn.Module):
    def __init__(self):
        super(melanomaClassifier, self).__init__()
        self.block1 = self.conv_block(c_in=3, c_out=256, dropout=0.2, kernel_size=5, stride=1, padding=2)
        self.block2 = self.conv_block(c_in=256, c_out=128, dropout=0.2, kernel_size=3, stride=1, padding=1)
        self.block3 = self.conv_block(c_in=128, c_out=64, dropout=0.2, kernel_size=3, stride=1, padding=1)
        self.lastcnn = nn.Conv2d(in_channels=64, out_channels=2, kernel_size=16, stride=1, padding=0)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.block1(x)
        x = self.maxpool(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.maxpool(x)
        x = self.lastcnn(x)
        return x
    def conv_block(self, c_in, c_out, dropout,  **kwargs):
        seq_block = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU(),
            nn.Dropout2d(p=dropout)
        )
        return seq_block

In [ ]:
model = melanomaClassifier()
model.to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.008)

In [ ]:
# weights = list(model.parameters())
# weights

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_tag, dim = 1)
    correct_results_sum = (y_pred_tags == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [ ]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [ ]:
print("Begin training.")
for e in tqdm(range(1, 21)):
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        y_train_pred = model(X_train_batch).squeeze()
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = binary_acc(y_train_pred, y_train_batch)
        train_loss.backward()
        optimizer.step()
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
    # VALIDATION
    with torch.no_grad():
        model.eval()
        val_epoch_loss = 0
        val_epoch_acc = 0
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            y_val_pred = model(X_val_batch).squeeze()
            y_val_pred = torch.unsqueeze(y_val_pred, 0)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = binary_acc(y_val_pred, y_val_batch)
            val_epoch_loss += train_loss.item()
            val_epoch_acc += train_acc.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
    print(f'Epoch {e+0:02}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

In [ ]:
train_val_acc_df = pd.DataFrame.from_dict(accuracy_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_loss_df = pd.DataFrame.from_dict(loss_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,10))
sns.lineplot(data=train_val_acc_df, x = "epochs", y="value", hue="variable",  ax=axes[0]).set_title('Train-Val Accuracy/Epoch')
sns.lineplot(data=train_val_loss_df, x = "epochs", y="value", hue="variable", ax=axes[1]).set_title('Train-Val Loss/Epoch')

In [ ]:
y_pred_list = []
y_true_list = []
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.cpu().numpy())

In [ ]:
y_pred_list = [i[0][0][0] for i in y_pred_list]
y_true_list = [i[0] for i in y_true_list]

In [ ]:
print(classification_report(y_true_list, y_pred_list))

In [ ]:
print(confusion_matrix(y_true_list, y_pred_list))


In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true_list, y_pred_list)).rename(columns=idx2class, index=idx2class)
fig, ax = plt.subplots(figsize=(7,5))         
sns.heatmap(confusion_matrix_df, annot=True, ax=ax)